# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download(['punkt', 'wordnet', 'stopwords'])
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier # fitting one classifier per each target 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # predictor 
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse_cleaned.db')
df = pd.read_sql_table('DisasterResponse_cleaned', con=engine)

category_names = list(set(df.columns) - set(['message', 'id', 'original', 'genre']))
X = df['message']  # it should be series for CountVectorizer input!!!!!!!
Y = df[category_names]

print(category_names)

['death', 'fire', 'direct_report', 'water', 'medical_products', 'military', 'medical_help', 'transport', 'electricity', 'buildings', 'shops', 'security', 'storm', 'food', 'related', 'other_weather', 'missing_people', 'other_aid', 'request', 'clothing', 'aid_centers', 'weather_related', 'earthquake', 'shelter', 'floods', 'cold', 'offer', 'refugees', 'money', 'other_infrastructure', 'aid_related', 'search_and_rescue', 'infrastructure_related', 'tools', 'hospitals']


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Replace Punctuations with a whitespace
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize with lemmatizing and stopwords removal  
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t.lower().strip()) for t in word_tokenize(text) \
              if t not in stopwords.words("english")]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class Debug(BaseEstimator, TransformerMixin): 
    def transform(self, X): 
        print(X.shape)
        return X 
    def fit(self, X, y=None, **fit_params): 
        return self

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('predictor', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
def generate_classification_report(model, X_test, Y_test, category_names):
    # inference on the test dataset
    Y_pred = pd.DataFrame(model.predict(X_test), columns=Y_test.columns)
    
    # generate the report 
    df_report = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1-score', \
                                      'distribution-0', 'distribution-1'])
    for col in Y_test.columns:
        # accuracy score from sklearn 'accruacy_score'
        accuracy = accuracy_score(Y_test[col], Y_pred[col])
        # other scores from sklearn 'classification_report'
        sample_weight = compute_sample_weight('balanced', Y_test[col]) # imbalanced dataset 
        report_str = classification_report(Y_test[col], Y_pred[col], sample_weight=sample_weight)
        metrics = report_str.split('\n')[5].split()[3:-1] 
        metrics = [float(m) for m in metrics] # flot conversion   
        # counts the class
        distributions = Y_test[col].value_counts().values / Y_test[col].shape[0]
        # append the results to the df_Report
        df_report.loc[col,:] =[accuracy] + metrics + distributions.tolist() 
    
    # add an average metrics of all categories 
    df_report.loc['avg',:] = df_report.mean()
    
    # print the report to the terminal 
    print(df_report.loc[category_names+['avg']]) 
    print(model.best_params_) 
    
    # save the report to the markdown table text
    with open('eval_report_markdown.txt', 'wt') as mf:
        fmt = ['---' for i in range(len(df_report.columns))]
        df_fmt = pd.DataFrame([fmt], columns=df_report.columns)
        df_formatted = pd.concat([df_fmt, df_report])
        markdown_str = df_formatted.to_csv(sep="|", index=False)
        mf.write(markdown_str)
    print('Evaluation report is generated in models/eval_report_markdown.txt')

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()
# pipeline.get_params().items()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'predictor', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'predictor__estimator__C', 'predictor__estimator__class_weight', 'predictor__estimator__dual', 'predictor__estimator__fit_intercept', 'predictor__estimator__intercept_scaling', 'predictor__estimator__max_iter', 'predictor__estimator__multi_class', 'predictor__estimator__n_jobs', 'predictor__estimator__penalty', 'predictor__estimator__random_state', 'predictor__estimator__solver', 'predictor__estimator__tol', 'predictor__estimator__verbose', 'predictor__estimator__warm_start', 'predictor__estimator', 'predictor__n_jobs'])

In [12]:
parameters = dict(
#     tfidf__norm=['l1'],

    predictor__estimator__random_state=[13], 
    
    # RandomForestClassifier
#     predictor__estimator__criterion=['entropy'], 

    # Rogistic regression
    predictor__estimator__solver=['liblinear'], #['liblinear', 'lbfgs', 'saga'],
    predictor__estimator__max_iter = [100],
    predictor__estimator__class_weight = [None], #[None, 'balanced'], 
# #     predictor__estimator__penalty = ['l2', 'elasticnet', 'none']
)

cv = GridSearchCV(pipeline, parameters,return_train_score =True)
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'predictor__estimator__random_state': [13], 'predictor__estimator__solver': ['liblinear'], 'predictor__estimator__max_iter': [100], 'predictor__estimator__class_weight': [None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.cv_results_

{'mean_fit_time': array([ 67.30714027]),
 'std_fit_time': array([ 1.66976154]),
 'mean_score_time': array([ 32.12772679]),
 'std_score_time': array([ 0.31852083]),
 'param_predictor__estimator__class_weight': masked_array(data = [None],
              mask = [False],
        fill_value = ?),
 'param_predictor__estimator__max_iter': masked_array(data = [100],
              mask = [False],
        fill_value = ?),
 'param_predictor__estimator__random_state': masked_array(data = [13],
              mask = [False],
        fill_value = ?),
 'param_predictor__estimator__solver': masked_array(data = ['liblinear'],
              mask = [False],
        fill_value = ?),
 'params': [{'predictor__estimator__class_weight': None,
   'predictor__estimator__max_iter': 100,
   'predictor__estimator__random_state': 13,
   'predictor__estimator__solver': 'liblinear'}],
 'split0_test_score': array([ 0.28006112]),
 'split1_test_score': array([ 0.26111536]),
 'split2_test_score': array([ 0.27563025]),
 'me

In [14]:
cv.best_params_

{'predictor__estimator__class_weight': None,
 'predictor__estimator__max_iter': 100,
 'predictor__estimator__random_state': 13,
 'predictor__estimator__solver': 'liblinear'}

In [36]:
generate_classification_report(cv, X_test, Y_test, category_names)

                        accuracy precision    recall  f1-score distribution-0  \
death                   0.963025      0.78      0.62      0.55       0.955233   
fire                    0.990833      0.75      0.52      0.37        0.99068   
direct_report           0.865393      0.78       0.7      0.68       0.807334   
water                   0.961192      0.83      0.76      0.74       0.932315   
medical_products        0.952941      0.77      0.59      0.51       0.945913   
military                0.968526      0.75      0.55      0.44       0.966998   
medical_help            0.927273      0.76      0.59      0.51       0.918717   
transport               0.957066      0.75      0.54      0.41       0.954469   
electricity             0.981513      0.76      0.55      0.45       0.980749   
buildings               0.954163      0.77       0.6      0.53       0.948358   
shops                     0.9945      0.25       0.5      0.33         0.9945   
security                0.98

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

1. solve the skewness of the data itself 
2. add a custum transformer, ~ action word extractor? 
3. This is a document level task, no need to consider wor2vec feature 

### 9. Export your model as a pickle file

In [ ]:
import pickle 
filename = 'disaster_reponse_final.pkl'
pickle.dump(cv, open(filename, 'wb')) 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.